In [2]:
import nltk
from nltk.collocations import *
import pymorphy2

bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

morph = pymorphy2.MorphAnalyzer()
punct = '.,!?():;'


In [7]:
words = [word.strip(punct) for word in open('court-V-N.csv').read().split()]
words_tagged = [morph.parse(word)[0].normal_form for word in words]
words_tagged = [x for x in words_tagged if x] # remove empty strings

words_tagged


['суд',
 'арестовать',
 'подполковник',
 'замначальник',
 'отдел',
 'суд',
 'вести',
 'переговоры',
 'поставка',
 'будущий',
 'суд',
 'взять',
 'реализация',
 'закон',
 'обязательный',
 'суд',
 'выкупить',
 'государство',
 'акция',
 'ранее',
 'суд',
 'говорить',
 'пресс-секретарь',
 'евгений',
 'временить',
 'суд',
 'есть',
 'звание',
 'заслужить',
 'машиностроитель',
 'суд',
 'заявить',
 'энергия',
 'проводить',
 'несколько',
 'суд',
 'изменить',
 'ситуация',
 'период',
 'немецкий',
 'суд',
 'напомнить',
 'требование',
 'объяснять',
 'соглашение',
 'суд',
 'обвинить',
 'работодатель',
 'применение',
 'серый',
 'суд',
 'обвинять',
 'дезинформация',
 'суд',
 'адвокат',
 'суд',
 'обращать',
 'внимание',
 'выступление',
 'иметь',
 'суд',
 'обсудить',
 'ход',
 'думский',
 'кампания',
 'суд',
 'объединить',
 'компания',
 'растить',
 'растить',
 'суд',
 'ожидать',
 'связь',
 'изменение',
 'вкалывать',
 'суд',
 'определиться',
 'враг',
 'народ',
 'коллаж',
 'суд',
 'осудить',
 'полковник',
 '

In [8]:
finder = BigramCollocationFinder.from_words(words_tagged)

In [9]:
finder.nbest(bigram_measures.pmi, 10)

[('арбитраж', 'встретиться'),
 ('армия', 'восстановиться'),
 ('аэропорт', 'расторжение'),
 ('балтика', 'нарушить'),
 ('банковский', 'комиссия'),
 ('безоговорочно', 'уступить'),
 ('бессмысленность', 'мероприятие'),
 ('бригадир', 'виктор'),
 ('бытность', 'министр'),
 ('вкладчик', 'ранний')]

In [10]:
finder.apply_freq_filter(3)
finder.nbest(bigram_measures.pmi, 10)

[('усилить', 'катастрофа'),
 ('надзорный', 'орган'),
 ('оператор', 'автомобильный'),
 ('таможенник', 'недействительный'),
 ('фактический', 'участник'),
 ('правота', 'надзорный'),
 ('прекратить', 'производство'),
 ('пресечение', 'вид'),
 ('освободить', 'залог'),
 ('отказ', 'регистрация')]

In [27]:
stopwords = nltk.corpus.stopwords.words('russian')
finder.apply_word_filter(lambda w: len(w) < 3 or w.lower() in stopwords)

res1 = finder.nbest(bigram_measures.likelihood_ratio, 10)

res_1 = []
for res in res1:
    res = ' '.join(res)
    res_1.append(res)
    
res_1

['суд суд',
 'суд признать',
 'выдать санкция',
 'суд удовлетворить',
 'удовлетворить иск',
 'удовлетворить ходатайство',
 'принять решение',
 'санкция арест',
 'мера пресечение',
 'наложить арест']

In [26]:
res2 = finder.nbest(bigram_measures.student_t, 10)

res_2 = []
for res in res2:
    res = ' '.join(res)
    res_2.append(res)
    
res_2

['суд признать',
 'суд удовлетворить',
 'суд вынести',
 'удовлетворить иск',
 'принять решение',
 'суд принять',
 'удовлетворить ходатайство',
 'суд отказать',
 'суд приговорить',
 'суд рассмотреть']

In [29]:
from nltk.metrics.spearman import *

with open('golden_standard.txt', 'r', encoding = 'utf-8') as f:
    gold = f.read().split('\n')
    
gold # золотой стандарт

['принять решение',
 'обратиться суд',
 'вынести приговор',
 'подать суд',
 'вынести решение',
 'просить суд',
 'удовлетворить иск',
 'признать виновный',
 'обязать опровергнуть',
 'обязать опубликовать']

In [36]:
res_1_ranks = list(ranks_from_sequence(res_1)) # likelihood ratio
res_2_ranks = list(ranks_from_sequence(res_2)) # student test
gold_ranks = list(ranks_from_sequence(gold))

print('%0.1f' % spearman_correlation(gold_ranks, res_1_ranks))
print('%0.1f' % spearman_correlation(gold_ranks, res_2_ranks))

-39.0
-24.0


Золотой стандарт был составлен с помощью топ-коллокаций (используя http://corpus.leeds.ac.uk/ruscorpora.html) и проверки их частотности в Google.

Топ-10 список res2 (student test) содержал 8/10 полных или частичных (1-словных) совпадений с вхождениями золотого стандарта. 

Топ-10 список res1 (log likelihood) содержал 5/10 полных или частичных совпадений с золотым стандартом.

Единственные полные совпадения в res1 и res2: 'удовлетворить иск' и 'принять решение'.

В топ-списке res2 в 7/10 вхождениях одним элементом биграммы является само слово 'суд', видимо из-за того, что (в отличие от log likelihood -- res1) метрика не учитывала, что 'суд' присутствовал в каждой строке файла 'court-V-N.csv'. 

Интересно, что и в golden standard и в res1 'суд' встретился ровно в 3 биграммах, но это скорее всего просто совпадение.

Также можно заметить, что в res2 все результаты -- сочетания VN или NV, тогда так в res1 присутствуют и NP ('санкция арест', 'мера пресечение', 'суд суд'), что не указывает именно на "действия".


In [37]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://raw.githubusercontent.com/phuuda/nlp_year4/master/hw3/results.png")